In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_claims = pd.read_csv('data/Claims.csv')
df_members = pd.read_csv('data/Members.csv')
df_hospital = pd.read_csv('data/DaysInHospital_Y2.csv')
df_drug = pd.read_csv('data/DrugCount.csv')
df_lab = pd.read_csv('data/LabCount.csv')

In [3]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                  object
LengthOfStay              object
DSFS                      object
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [4]:
df_claims

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0
5,45844561,1963488.0,4042.0,55823.0,Y3,Pediatrics,Office,25,NaN,3- 4 months,NEUMENT,0,EM,0
6,99829076,6721023.0,265273.0,91972.0,Y1,Rehabilitation,Office,162+,NaN,0- 1 month,TRAUMA,0,MED,0
7,54666321,9932074.0,35565.0,27294.0,Y1,Diagnostic Imaging,Office,29,NaN,0- 1 month,RESPR4,0,RAD,0
8,60497718,363858.0,293107.0,64913.0,Y2,Rehabilitation,Office,42,NaN,1- 2 months,INFEC4,0,EM,0
9,72200595,6251259.0,791272.0,49465.0,Y3,Internal,Office,56,NaN,7- 8 months,MISCHRT,1-2,SCS,0


In [5]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['count','percent']
    display(null_df)
    
count_nulls(df_claims)

,count,percent
ProviderID,16264,0.609369
Vendor,24856,0.931289
PCP,7492,0.280705
Specialty,8405,0.314913
PlaceSvc,7632,0.285951
LengthOfStay,2597392,97.317412
DSFS,52770,1.977152
PrimaryConditionGroup,11410,0.427503
ProcedureGroup,3675,0.137693


In [6]:
def updatePayDelay():
    df_claims.loc[df_claims['PayDelay']=='162+','PayDelay'] = 162
    df_claims['PayDelay'] = df_claims['PayDelay'].astype(int)
    return

updatePayDelay()

In [7]:
def updateDSFS():
    df_claims.loc[df_claims['DSFS']=='0- 1 month','DSFS'] = 1
    df_claims.loc[df_claims['DSFS']=='1- 2 months','DSFS'] = 2
    df_claims.loc[df_claims['DSFS']=='2- 3 months','DSFS'] = 3
    df_claims.loc[df_claims['DSFS']=='3- 4 months','DSFS'] = 4
    df_claims.loc[df_claims['DSFS']=='4- 5 months','DSFS'] = 5
    df_claims.loc[df_claims['DSFS']=='5- 6 months','DSFS'] = 6
    df_claims.loc[df_claims['DSFS']=='6- 7 months','DSFS'] = 7
    df_claims.loc[df_claims['DSFS']=='7- 8 months','DSFS'] = 8
    df_claims.loc[df_claims['DSFS']=='8- 9 months','DSFS'] = 9
    df_claims.loc[df_claims['DSFS']=='9-10 months','DSFS'] = 10
    df_claims.loc[df_claims['DSFS']=='10-11 months','DSFS'] = 11
    df_claims.loc[df_claims['DSFS']=='11-12 months','DSFS'] = 12
    df_claims['DSFS'].fillna(0, inplace=True)
    df_claims['DSFS'] = df_claims['DSFS'].astype(int)
    return

updateDSFS()

In [8]:
def updateCharlsonIndex():
    df_claims.loc[df_claims['CharlsonIndex']=='0','CharlsonIndex'] = 0
    df_claims.loc[df_claims['CharlsonIndex']=='1-2','CharlsonIndex'] = 2
    df_claims.loc[df_claims['CharlsonIndex']=='3-4','CharlsonIndex'] = 4
    df_claims.loc[df_claims['CharlsonIndex']=='5+','CharlsonIndex'] = 6

    df_claims['CharlsonIndex'] = df_claims['CharlsonIndex'].astype(int)
    return
    
updateCharlsonIndex()

In [9]:
def updateLengthOfStay():
    df_claims.loc[df_claims['LengthOfStay']=='1 day','LengthOfStay'] = 1
    df_claims.loc[df_claims['LengthOfStay']=='2 days','LengthOfStay'] = 2
    df_claims.loc[df_claims['LengthOfStay']=='3 days','LengthOfStay'] = 3
    df_claims.loc[df_claims['LengthOfStay']=='4 days','LengthOfStay'] = 4
    df_claims.loc[df_claims['LengthOfStay']=='5 days','LengthOfStay'] = 5
    df_claims.loc[df_claims['LengthOfStay']=='6 days','LengthOfStay'] = 6
    df_claims.loc[df_claims['LengthOfStay']=='1- 2 weeks','LengthOfStay'] = 11
    df_claims.loc[df_claims['LengthOfStay']=='2- 4 weeks','LengthOfStay'] = 21
    df_claims.loc[df_claims['LengthOfStay']=='4- 8 weeks','LengthOfStay'] = 42
    df_claims.loc[df_claims['LengthOfStay']=='26+ weeks','LengthOfStay'] = 180
    
    df_claims['LengthOfStay'].fillna(0, inplace=True)

    df_claims['LengthOfStay'] = df_claims['LengthOfStay'].astype(int)
    return

updateLengthOfStay()

In [11]:
df_group = df_claims.groupby(['Year','MemberID'])

df_claims['no_Claims'] = df_group['ProviderID'].transform('count')
df_claims['no_Providers'] = df_group['ProviderID'].transform('nunique')
df_claims['no_Vendors'] = df_group['Vendor'].transform('nunique')
df_claims['no_PCPs'] = df_group['PCP'].transform('nunique')
df_claims['no_PlaceSvcs'] = df_group['PlaceSvc'].transform('nunique')
df_claims['no_Specialities'] = df_group['Specialty'].transform('nunique')
df_claims['no_PrimaryConditionGroups'] = df_group['PrimaryConditionGroup'].transform('nunique')
df_claims['no_ProcedureGroups'] = df_group['ProcedureGroup'].transform('nunique')

df_claims['PayDelay_max'] = df_group['PayDelay'].transform('max')
df_claims['PayDelay_min'] = df_group['PayDelay'].transform('min')
df_claims['PayDelay_ave'] = df_group['PayDelay'].transform(lambda x: np.average(x))
# df_claims['PayDelay_stdev'] = df_group['PayDelay'].transform(lambda x: 1 if x>25 else 0)

df_claims['pgc1'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MSC2a3')) 
df_claims['pgc2'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB3')) 
df_claims['pgc3'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ARTHSPIN')) 
df_claims['pgc4'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='NEUMENT')) 
df_claims['pgc5'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RESPR4')) 
df_claims['pgc6'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCHRT')) 
df_claims['pgc7'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SKNAUT')) 
df_claims['pgc8'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIBLEED')) 
df_claims['pgc9'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='INFEC4')) 
df_claims['pgc10'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='TRAUMA')) 
df_claims['pgc11'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART2')) 
df_claims['pgc12'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL3')) 
df_claims['pgc13'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ROAMI')) 
df_claims['pgc14'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL5')) 
df_claims['pgc15'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ODaBNCA')) 
df_claims['pgc16'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='UTI')) 
df_claims['pgc17'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='COPD'))
df_claims['pgc18'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNEC1')) 
df_claims['pgc19'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRB')) 
df_claims['pgc20'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FXDISLC')) 
df_claims['pgc21'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='AMI')) 
df_claims['pgc22'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PRGNCY')) 
df_claims['pgc23'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEMTOL')) 
df_claims['pgc24'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART4')) 
df_claims['pgc25'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEIZURE')) 
df_claims['pgc26'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='APPCHOL')) 
df_claims['pgc27'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CHF')) 
df_claims['pgc28'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNECA')) 
df_claims['pgc29'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x.isnull())) 
df_claims['pgc30'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNEUM')) 
df_claims['pgc31'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL2')) 
df_claims['pgc32'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIOBSENT')) 
df_claims['pgc33'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='STROKE')) 
df_claims['pgc34'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRA')) 
df_claims['pgc35'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FLaELEC')) 
df_claims['pgc36'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL1')) 
df_claims['pgc37'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HIPFX')) 
df_claims['pgc38'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB1')) 
df_claims['pgc39'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERVALV')) 
df_claims['pgc40'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='LIVERDZ'))
df_claims['pgc41'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CATAST')) 
df_claims['pgc42'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRM')) 
df_claims['pgc43'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERINTL')) 
df_claims['pgc44'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNCRDZ')) 
df_claims['pgc45'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL1')) 
df_claims['pgc46'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEPSIS')) 

df_claims['sp1'] = df_group['Specialty'].transform(lambda x: sum(x=='Internal')) 
df_claims['sp2'] = df_group['Specialty'].transform(lambda x: sum(x=='Laboratory')) 
df_claims['sp3'] = df_group['Specialty'].transform(lambda x: sum(x=='General Practice)) 
df_claims['sp4'] = df_group['Specialty'].transform(lambda x: sum(x=='Surgery')) 
df_claims['sp5'] = df_group['Specialty'].transform(lambda x: sum(x=='Diagnostic Imaging')) 
df_claims['sp6'] = df_group['Specialty'].transform(lambda x: sum(x=='Emergency')) 
df_claims['sp7'] = df_group['Specialty'].transform(lambda x: sum(x=='Other')) 
df_claims['sp8'] = df_group['Specialty'].transform(lambda x: sum(x=='Pediatrics')) 
df_claims['sp9'] = df_group['Specialty'].transform(lambda x: sum(x=='Rehabilitation')) 
df_claims['sp10'] = df_group['Specialty'].transform(lambda x: sum(x=='Obstetrics and Gynecology')) 
df_claims['sp11'] = df_group['Specialty'].transform(lambda x: sum(x=='Anesthesiology')) 
df_claims['sp12'] = df_group['Specialty'].transform(lambda x: sum(x=='Pathology')) 
df_claims['sp13'] = df_group['Specialty'].transform(lambda x: sum(x.isnull()))

df_claims['pg1'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='EM'))
df_claims['pg2'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='PL'))
df_claims['pg3'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='MED'))                                                                 
df_claims['pg4'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SCS'))
df_claims['pg5'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='RAD'))
df_claims['pg6'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SDS'))
df_claims['pg7'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SIS'))
df_claims['pg8'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMS'))                                                                 
df_claims['pg9'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='ANES'))
df_claims['pg10'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SGS'))
df_claims['pg11'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SEOA'))
df_claims['pg12'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SRS'))
df_claims['pg13'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SNS'))                                                                 
df_claims['pg14'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SAS'))
df_claims['pg15'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SUS'))
df_claims['pg16'] = df_group['ProcedureGroup'].transform(lambda x: sum(x.isnull()))
df_claims['pg17'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMCD'))
df_claims['pg18'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SO'))                                                                 


In [14]:
df_claims

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,...,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,PayDelay_max,PayDelay_min,PayDelay_ave,pgc1,pgc2
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,0,9,...,1.0,3,5,8,5,114,27,49.416667,2,2
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,0,8,...,1.0,3,7,8,7,135,0,27.750000,7,6
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,0,1,...,1.0,2,2,1,3,31,14,25.333333,0,3
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,0,6,...,1.0,2,5,7,6,86,15,34.733333,3,13
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,0,5,...,1.0,3,6,7,5,42,15,29.116279,17,2
5,45844561,1963488.0,4042.0,55823.0,Y3,Pediatrics,Office,25,0,4,...,1.0,1,1,2,1,25,17,20.166667,1,0
6,99829076,6721023.0,265273.0,91972.0,Y1,Rehabilitation,Office,162,0,1,...,1.0,2,5,4,5,162,23,106.000000,2,0
7,54666321,9932074.0,35565.0,27294.0,Y1,Diagnostic Imaging,Office,29,0,1,...,1.0,2,6,5,6,162,20,57.750000,15,0
8,60497718,363858.0,293107.0,64913.0,Y2,Rehabilitation,Office,42,0,2,...,1.0,3,4,8,5,162,21,45.066667,3,0
9,72200595,6251259.0,791272.0,49465.0,Y3,Internal,Office,56,0,8,...,1.0,6,8,14,7,162,0,51.954545,3,1
